In [22]:
import torch
from utils.fda import FDA_source_to_target
from utils.aug import ExtCompose, ExtToTensor, ExtRandomHorizontalFlip , ExtScale , ExtRandomCrop, ExtGaussianBlur, ExtColorJitter, ExtRandomCompose, ExtResize, ExtNormalize
from datasets.cityscapes import CityScapes
from datasets.GTA5 import GTA5
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

## FDA Style Transfer Test

In [23]:
beta = 0.05
h = 512
w = 1024
b = (  np.floor(np.amin((h,w))*beta)  ).astype(int)
b

25

In [21]:
standard_transformations = ExtCompose([ExtScale(0.5,interpolation=Image.Resampling.BILINEAR), ExtToTensor()])
transformations = ExtCompose([ExtResize((512,1024)), ExtToTensor()])
target_transformations = standard_transformations

train_dataset = GTA5(root='dataset',transforms=transformations)
target_dataset_train = CityScapes(split='train',transforms=target_transformations)

source_dataloader_train = DataLoader(train_dataset,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)
target_dataloader_train = DataLoader(target_dataset_train,
                batch_size=1,
                shuffle=False,
                num_workers=1,
                pin_memory=False,
                drop_last=True)

for i, (source_data, target_data) in enumerate(zip(source_dataloader_train,target_dataloader_train)):
  
    if i > 5:
        break

    source_data,source_label = source_data
    target_data,_ = target_data
    source_data = source_data.cuda()
    source_label = source_label.long().cuda()
    target_data = target_data.cuda()

    t_source_data = FDA_source_to_target(source_data, target_data, beta=beta)

    # Visualize the image (.png) and the transformed image (.jpg)
    Image.fromarray(target_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    Image.fromarray(source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()
    Image.fromarray(t_source_data.detach().cpu().numpy().astype('uint8')[0].transpose(1, 2, 0)).show()


root:  dataset
